In [1]:
from astropy.io import fits
from pathlib import Path
from astropy.time import Time
import pandas as pd
import numpy as np
from astropy.table import Table
from obsutils.utils import SkyportalClient
from io import StringIO

In [2]:
base_dir = Path.home() / "Data/ldt/20250518/redux"

send_to_fritz = True

instrument_id = 46 # LDT is 46

In [3]:
files = list(base_dir.glob("*_tellcorr.fits"))

In [4]:
for path in files:
    with fits.open(path) as hdul:
        source = hdul[0].header["TARGET"]
        date = Time(hdul[0].header["DATE-OBS"], format="isot")
        data = hdul[1].data
        
    df = Table(data).to_pandas()[["wave", "flux", "sigma"]]
    df.rename(columns={"wave": "#wl"}, inplace=True)
    
    # Fix for SNID
    df = df[df["flux"] > 0]
    outpath = path.parent / f"clean_{path.with_suffix(".txt").name}"
    print(f"Saving to {outpath}")
    df.to_csv(outpath, sep=" ", index=False)
    
    if send_to_fritz:
        
        text = df.to_string()
        string_buffer = StringIO()
    
        df.to_csv(string_buffer, index=False)
        text = string_buffer.getvalue()
    
        filename = f"LDT_{str(date.isot).split('T')[0].replace('-', '')}_{path.name}"
        
        payload = {
            "wave_column": 0,
            "flux_column": 1,
            "fluxerr_column": 2,
            "observed_at": str(date),
            "obj_id": source,
            "instrument_id": instrument_id,
            "ascii": text,
            "filename": path.name,
            "group_ids": [1]
        }
    
        client = SkyportalClient()
        client.set_up_session()
    
        res = client.api(
            "GET", 
            f"sources/{source}/spectra",
        )
        existing_spectra = res.json()["data"]["spectra"]
        filenames = [x["original_file_filename"] for x in existing_spectra]
    
        if path.name in filenames:
            print(f"Spectrum {filename} already uploaded.")
        else:
            print(f"Uploading spectrum {filename}.")
            res = client.api(
                "POST", 
                "spectra/ascii",
                data=payload
            )
            res.raise_for_status()

Saving to /Users/robertstein/Data/ldt/20250518/redux/clean_ZTF25aapwprt_tellcorr.txt
Uploading spectrum LDT_20250519_ZTF25aapwprt_tellcorr.fits.
Saving to /Users/robertstein/Data/ldt/20250518/redux/clean_ZTF18aaotzun_tellcorr.txt
Uploading spectrum LDT_20250519_ZTF18aaotzun_tellcorr.fits.
Saving to /Users/robertstein/Data/ldt/20250518/redux/clean_ZTF20aappkya_tellcorr.txt
Uploading spectrum LDT_20250519_ZTF20aappkya_tellcorr.fits.
Saving to /Users/robertstein/Data/ldt/20250518/redux/clean_ZTF25aaooewz_tellcorr.txt
Spectrum LDT_20250519_ZTF25aaooewz_tellcorr.fits already uploaded.
